In [6]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs\Patos\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'DATA (YYYY-MM-DD)',
    'HORA (UTC)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas
        renomear = {
            'DATA (YYYY-MM-DD)': 'data',
            'HORA (UTC)': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Substitui TODAS as variações de -9999 por NaN (strings e números) ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace(-9999, np.nan)
        df = df.replace(-9999.0, np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # --- Segurança extra: garante que ainda não exista -9999 em float ---
        df.loc[df['temp'] == -9999, 'temp'] = np.nan
        df.loc[df['umidade'] == -9999, 'umidade'] = np.nan
        df.loc[df['pressao'] == -9999, 'pressao'] = np.nan

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [
            f"{var}_{estat}"
            for var, estat in agrupado.columns
        ]

        # Reset index para voltar a ter 'data' como coluna
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_patos1 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_patos1['data'] = pd.to_datetime(df_patos1['data'], errors='coerce')
df_patos1.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_patos1 = df_patos1[[col for col in colunas_final if col in df_patos1.columns]]

# Exibe amostra
print(df_patos1.head(10))


           data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
4018 2007-07-21      20.0      31.8  26.333333         36.0         82.0   
4019 2007-07-22      22.6      27.6  24.883333         49.0         93.0   
4020 2007-07-23      22.5      30.1  25.441667         44.0         88.0   
4021 2007-07-24      20.1      33.3  26.508333         28.0         92.0   
4022 2007-07-25      21.2      32.4  26.820833         31.0         79.0   
4023 2007-07-26      21.5      34.1  27.350000         30.0         84.0   
4024 2007-07-27      22.9      32.3  26.912500         33.0         78.0   
4025 2007-07-28      20.9      32.7  26.473913         29.0         66.0   
4026 2007-07-29      21.1      33.1  26.720833         29.0         77.0   
4027 2007-07-30      21.6      33.9  27.291667         18.0         69.0   

      umidade_mean  pressao_min  pressao_max  pressao_mean  
4018     53.041667        982.8        987.0    984.912500  
4019     72.000000        981.8        98

In [37]:
print(df_patos1.head(50))


           data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
4018 2007-07-21      20.0      31.8  26.333333         36.0         82.0   
4019 2007-07-22      22.6      27.6  24.883333         49.0         93.0   
4020 2007-07-23      22.5      30.1  25.441667         44.0         88.0   
4021 2007-07-24      20.1      33.3  26.508333         28.0         92.0   
4022 2007-07-25      21.2      32.4  26.820833         31.0         79.0   
4023 2007-07-26      21.5      34.1  27.350000         30.0         84.0   
4024 2007-07-27      22.9      32.3  26.912500         33.0         78.0   
4025 2007-07-28      20.9      32.7  26.473913         29.0         66.0   
4026 2007-07-29      21.1      33.1  26.720833         29.0         77.0   
4027 2007-07-30      21.6      33.9  27.291667         18.0         69.0   
4028 2007-07-31      22.1      31.7  26.187500         33.0         74.0   
4029 2007-08-01      22.2      31.1  26.170833         38.0         75.0   
4030 2007-08

In [39]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs2\Patos\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'Data',
    'HORA UTC',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas (ajuste no nome exato de 'HORA UTC')
        renomear = {
            'Data': 'data',
            'HORA UTC': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Trata coluna 'hora' no formato "0000 UTC" ---
        if 'hora' in df.columns:
            df['hora'] = (
                df['hora']
                .astype(str)
                .str.extract(r'(\d{4})')[0]  # extrai os 4 dígitos
                .apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) and len(x) == 4 else np.nan)
            )

        # --- Substitui -9999 e células vazias por NaN ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace('', np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [f"{var}_{estat}" for var, estat in agrupado.columns]
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_patos2 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_patos2['data'] = pd.to_datetime(df_patos2['data'], errors='coerce')
df_patos2.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_patos2 = df_patos2[[col for col in colunas_final if col in df_patos2.columns]]

# Exibe amostra
print(df_patos2.head(10))


        data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
0 2019-01-01      24.0      37.4  30.437500         10.0         57.0   
1 2019-01-02      25.4      37.9  30.637500         10.0         52.0   
2 2019-01-03      23.6      36.4  29.766667         10.0         56.0   
3 2019-01-04      23.6      36.3  28.800000         11.0         70.0   
4 2019-01-05      23.0      35.6  27.958333         15.0         80.0   
5 2019-01-06      24.1      33.2  27.820833         18.0         77.0   
6 2019-01-07      24.4      34.0  28.116667         21.0         62.0   
7 2019-01-08      23.0      33.4  26.829167         23.0         84.0   
8 2019-01-09      22.2      33.8  26.879167         22.0         89.0   
9 2019-01-10      23.4      34.0  28.300000         14.0         61.0   

   umidade_mean  pressao_min  pressao_max  pressao_mean  
0     34.619048        980.5        985.5    982.979167  
1     32.761905        981.0        985.5    983.716667  
2     29.708333       

# Areia


In [12]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs\Areia\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'DATA (YYYY-MM-DD)',
    'HORA (UTC)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas
        renomear = {
            'DATA (YYYY-MM-DD)': 'data',
            'HORA (UTC)': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Substitui TODAS as variações de -9999 por NaN (strings e números) ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace(-9999, np.nan)
        df = df.replace(-9999.0, np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # --- Segurança extra: garante que ainda não exista -9999 em float ---
        df.loc[df['temp'] == -9999, 'temp'] = np.nan
        df.loc[df['umidade'] == -9999, 'umidade'] = np.nan
        df.loc[df['pressao'] == -9999, 'pressao'] = np.nan

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [
            f"{var}_{estat}"
            for var, estat in agrupado.columns
        ]

        # Reset index para voltar a ter 'data' como coluna
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_areia1 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_areia1['data'] = pd.to_datetime(df_areia1['data'], errors='coerce')
df_areia1.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_areia1 = df_areia1[[col for col in colunas_final if col in df_areia1.columns]]

# Exibe amostra
print(df_areia1.head(10))


        data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
0 2007-01-01      20.5      28.5  23.637500         57.0        100.0   
1 2007-01-02      21.2      26.3  22.891667         68.0        100.0   
2 2007-01-03      19.9      28.0  22.779167         48.0        100.0   
3 2007-01-04      19.7      27.1  22.495833         68.0        100.0   
4 2007-01-05      20.8      27.6  23.287500         58.0        100.0   
5 2007-01-06      20.5      27.0  22.858333         66.0        100.0   
6 2007-01-07      19.9      28.1  23.233333         57.0        100.0   
7 2007-01-08      20.9      26.9  23.000000         58.0        100.0   
8 2007-01-09      20.8      27.8  23.408333         65.0        100.0   
9 2007-01-10      20.7      28.4  23.541667         56.0        100.0   

   umidade_mean  pressao_min  pressao_max  pressao_mean  
0     82.750000        946.8        950.2    948.587500  
1     88.173913        947.6        950.5    949.029167  
2     74.153846       

In [10]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs2\Areia\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'Data',
    'HORA UTC',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas (ajuste no nome exato de 'HORA UTC')
        renomear = {
            'Data': 'data',
            'HORA UTC': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Trata coluna 'hora' no formato "0000 UTC" ---
        if 'hora' in df.columns:
            df['hora'] = (
                df['hora']
                .astype(str)
                .str.extract(r'(\d{4})')[0]  # extrai os 4 dígitos
                .apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) and len(x) == 4 else np.nan)
            )

        # --- Substitui -9999 e células vazias por NaN ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace('', np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [f"{var}_{estat}" for var, estat in agrupado.columns]
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_areia2 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_areia2['data'] = pd.to_datetime(df_areia2['data'], errors='coerce')
df_areia2.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_areia2 = df_areia2[[col for col in colunas_final if col in df_areia2.columns]]

# Exibe amostra
print(df_areia2.head(10))


        data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
0 2019-01-01      21.4      30.0  24.550000         49.0         94.0   
1 2019-01-02      21.6      30.3  24.700000         43.0         94.0   
2 2019-01-03      20.7      30.2  24.329167         44.0         94.0   
3 2019-01-04      20.6      30.3  24.287500         46.0         95.0   
4 2019-01-05      21.6      28.9  24.125000         52.0         93.0   
5 2019-01-06      21.2      28.6  24.016667         48.0         95.0   
6 2019-01-07      20.7      28.4  23.370833         56.0         94.0   
7 2019-01-08      20.5      27.8  23.062500         57.0         95.0   
8 2019-01-09      20.6      27.3  22.979167         57.0         96.0   
9 2019-01-10      20.7      29.7  24.000000         40.0         95.0   

   umidade_mean  pressao_min  pressao_max  pressao_mean  
0     77.125000        947.8        951.0    949.225000  
1     73.250000        948.6        951.4    950.012500  
2     74.125000       

# João Pessoa

In [21]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs\Joao Pessoa\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'DATA (YYYY-MM-DD)',
    'HORA (UTC)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas
        renomear = {
            'DATA (YYYY-MM-DD)': 'data',
            'HORA (UTC)': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Substitui TODAS as variações de -9999 por NaN (strings e números) ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace(-9999, np.nan)
        df = df.replace(-9999.0, np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # --- Segurança extra: garante que ainda não exista -9999 em float ---
        df.loc[df['temp'] == -9999, 'temp'] = np.nan
        df.loc[df['umidade'] == -9999, 'umidade'] = np.nan
        df.loc[df['pressao'] == -9999, 'pressao'] = np.nan

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [
            f"{var}_{estat}"
            for var, estat in agrupado.columns
        ]

        # Reset index para voltar a ter 'data' como coluna
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_jampa1 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_jampa1['data'] = pd.to_datetime(df_jampa1['data'], errors='coerce')
df_jampa1.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_jampa1 = df_jampa1[[col for col in colunas_final if col in df_jampa1.columns]]

# Exibe amostra
print(df_jampa1.head(10))


           data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
4018 2007-07-21      24.3      26.4  25.060000         70.0         84.0   
4019 2007-07-22      21.8      26.2  23.345833         74.0         95.0   
4020 2007-07-23      21.4      27.9  24.312500         60.0         94.0   
4021 2007-07-24      21.0      28.1  24.439130         60.0         93.0   
4022 2007-07-25      21.0      28.2  25.045833         62.0         95.0   
4023 2007-07-26      22.3      27.8  24.620833         71.0         94.0   
4024 2007-07-27      22.1      27.8  24.954167         62.0         96.0   
4025 2007-07-28      20.1      27.8  23.957143         59.0         94.0   
4026 2007-07-29      21.5      27.9  24.245833         61.0         94.0   
4027 2007-07-30      21.8      27.7  24.316667         59.0         89.0   

      umidade_mean  pressao_min  pressao_max  pressao_mean  
4018     79.000000       1009.3       1010.3   1009.700000  
4019     88.666667       1007.7       101

In [23]:
import pandas as pd
import glob
import numpy as np

# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs2\Joao Pessoa\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse
colunas_uteis = [
    'Data',
    'HORA UTC',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)'
]

dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(
            arquivo,
            encoding='latin1',
            delimiter=';',
            skiprows=8,
            low_memory=False
        )

        # Mantém apenas colunas desejadas
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia colunas (ajuste no nome exato de 'HORA UTC')
        renomear = {
            'Data': 'data',
            'HORA UTC': 'hora',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'pressao'
        }
        df.rename(columns=renomear, inplace=True)

        # --- Trata coluna 'hora' no formato "0000 UTC" ---
        if 'hora' in df.columns:
            df['hora'] = (
                df['hora']
                .astype(str)
                .str.extract(r'(\d{4})')[0]  # extrai os 4 dígitos
                .apply(lambda x: f"{x[:2]}:{x[2:]}" if pd.notna(x) and len(x) == 4 else np.nan)
            )

        # --- Substitui -9999 e células vazias por NaN ---
        df = df.replace(r'^\s*-?9999(\.0+)?\s*$', np.nan, regex=True)
        df = df.replace('', np.nan)

        # Converte colunas numéricas
        for col in ['temp', 'umidade', 'pressao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # Agrupa por dia e calcula estatísticas
        agrupado = df.groupby('data').agg({
            'temp': ['min', 'max', 'mean'],
            'umidade': ['min', 'max', 'mean'],
            'pressao': ['min', 'max', 'mean']
        })

        # Ajusta nomes das colunas (multiindex → plano)
        agrupado.columns = [f"{var}_{estat}" for var, estat in agrupado.columns]
        agrupado.reset_index(inplace=True)

        dfs.append(agrupado)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os arquivos
df_jampa2 = pd.concat(dfs, ignore_index=True)

# Converte 'data' para datetime e ordena
df_jampa2['data'] = pd.to_datetime(df_jampa2['data'], errors='coerce')
df_jampa2.sort_values(by='data', inplace=True)

# Reorganiza colunas
colunas_final = [
    'data',
    'temp_min', 'temp_max', 'temp_mean',
    'umidade_min', 'umidade_max', 'umidade_mean',
    'pressao_min', 'pressao_max', 'pressao_mean'
]
df_jampa2 = df_jampa2[[col for col in colunas_final if col in df_jampa2.columns]]

# Exibe amostra
print(df_jampa2.head(10))


        data  temp_min  temp_max  temp_mean  umidade_min  umidade_max  \
0 2019-01-01      25.0      30.8  27.358333         60.0         88.0   
1 2019-01-02      26.2      30.4  27.762500         58.0         82.0   
2 2019-01-03      25.7      30.8  27.754167         54.0         77.0   
3 2019-01-04      25.7      31.1  27.825000         57.0         80.0   
4 2019-01-05      26.5      31.1  27.904167         60.0         79.0   
5 2019-01-06      25.5      30.9  27.700000         55.0         80.0   
6 2019-01-07      23.2      31.0  27.058333         57.0         90.0   
7 2019-01-08      26.5      31.5  28.012500         55.0         76.0   
8 2019-01-09      26.5      31.8  28.216667         53.0         79.0   
9 2019-01-10      26.2      31.9  28.166667         53.0         76.0   

   umidade_mean  pressao_min  pressao_max  pressao_mean  
0     75.625000       1008.6       1012.1   1010.358333  
1     72.000000       1010.3       1012.5   1011.333333  
2     67.208333       

# Campina Grande

# Cabaceiras

# Monteiro

# Camaratuba

# São Gonçalo